<a href="https://colab.research.google.com/github/mathluva/Bert-Toxic-Comments/blob/main/Toxic_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Feb 17 01:01:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf

In [4]:
!git clone --depth 1 -b v2.3.0 https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 2650, done.
remote: Counting objects: 100% (2650/2650), done.
remote: Compressing objects: 100% (2311/2311), done.
remote: Total 2650 (delta 507), reused 1383 (delta 306), pack-reused 0
Receiving objects: 100% (2650/2650), 34.01 MiB | 34.35 MiB/s, done.
Resolving deltas: 100% (507/507), done.
Note: checking out '400d68abbccda2f0f6609e3a924467718b144233'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [5]:
# install requirements to use tensorflow/models repository
!pip install -Uqr models/official/requirements.txt
# you may have to restart the runtime afterwards

     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 215kB 29.4MB/s 
     |████████████████████████████████| 296kB 54.6MB/s 
     |████████████████████████████████| 102kB 16.0MB/s 
     |████████████████████████████████| 25.9MB 115kB/s 
     |████████████████████████████████| 174kB 59.3MB/s 
     |████████████████████████████████| 3.7MB 19.4MB/s 
     |████████████████████████████████| 706kB 50.7MB/s 
     |████████████████████████████████| 358kB 58.1MB/s 
     |████████████████████████████████| 1.2MB 53.2MB/s 
     |████████████████████████████████| 11.5MB 51.6MB/s 
     |████████████████████████████████| 37.6MB 78kB/s 
     |████████████████████████████████| 645kB 56.7MB/s 
     |████████████████████████████████| 2.2MB 46.8MB/s 
     |████████████████████████████████| 102kB 15.4MB/s 
     |████████████████████████████████| 51kB 9.7MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
ERROR: pandas-gbq 0.13.3 has requirement google-clou

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import sys
sys.path.append('models')
from official.nlp.data import classifier_data_lib
from official.nlp.bert import tokenization
from official.nlp import optimization

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
from google.colab import files
uploaded = files.upload()

Saving train_preprocessed.csv to train_preprocessed.csv


In [7]:
import io
df = pd.read_csv(io.BytesIO(uploaded['train_preprocessed.csv']))

In [8]:
df.shape

(159571, 10)

In [10]:
df.tail(20)

,comment_text,id,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity
159551,while about half the references are from byu i...,ffbc2db4225258dd,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
159552,prague spring i think that prague spring deser...,ffbcd64a71775e04,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
159553,i see this as having been merged undoing one ...,ffbd331a3aa269b9,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
159554,and i m going to keep posting the stuff u dele...,ffbdbb0483ed0841,0.0,1.0,1.0,train,0.0,0.0,1.0,3.0
159555,how come when you download that mp it s titl...,ffc2f409658571f1,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
159556,i ll be on irc too if you have a more specif...,ffc671f2acdd80e1,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
159557,it is my opinion that that happens to be off t...,ffc7bbb177c3c966,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
159558,please stop removing content from wikipedia i...,ffca1e81aefc48ac,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
159559,image barack obama mother jpg listed for delet...,ffca8d71d71a3fae,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
159560,editing of article without consensus removal ...,ffcdcb71854f6d8a,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0


In [ ]:
train_df, remaining